Reload Pytorch model generated by manifold.ipynb

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from ipywidgets import interact, IntSlider

import torch
from torch.autograd import Variable

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


# Reload model

In [2]:
H1, H2 = 200, 200

torch.manual_seed(7) # reproducible

model = torch.nn.Sequential(
    torch.nn.Linear(75, H1),
    torch.nn.Tanh(),
    torch.nn.Linear(H1, H2),
    torch.nn.Tanh(),
    torch.nn.Linear(H2, 74)
)

In [3]:
model.load_state_dict(torch.load('trained_model.pt'))

In [4]:
list(model.parameters())[0]

Parameter containing:
-1.9669e-03 -6.3742e-03 -1.6819e-03  ...   2.2053e-02 -3.3278e-02  6.3223e-02
-2.5824e-05  1.1929e-04 -1.2944e-05  ...  -2.5817e-03 -3.8335e-02 -4.4682e-01
-4.1317e-03 -4.3728e-03 -9.5799e-03  ...  -3.0486e-03  1.7924e-02  1.2579e-02
                ...                   ⋱                   ...                
-1.5980e-04 -7.2200e-04  3.5545e-04  ...  -3.2094e-02  1.8427e-02 -7.1764e-02
 1.0604e-02  1.1191e-02 -5.9377e-03  ...  -3.8323e-03 -9.4209e-04 -3.1001e-02
 1.1682e-03  3.9280e-04 -2.9998e-04  ...  -1.9707e-03  1.6533e-02 -4.1770e-01
[torch.FloatTensor of size 200x75]

In [5]:
df_y0 = pd.read_csv('train_data.csv')
df_y0.head()

,Unnamed: 0,H2SO4,HCOOH,CCO_OH,RCO_OH,CCO_OOH,RCO_OOH,XN,XC,SO2,...,C_O2,BZCO_O2,RO2_R,NO,NO2,NO3,CCO_O2,RCO_O2,OH,SUN
0,0,4.716944e+11,1.090939e+11,1.525966e+12,8.314627e+11,1.897158e+11,1.152369e+11,1.173206e+10,5.501899e+12,7.228313e+11,...,7.727887e+08,4.547033e+05,1.463542e+09,6.709683e+10,1.442070e+12,8.028878e+09,1.052131e+08,2.268188e+06,5.932402e+07,0.706857
1,1,8.979349e+11,9.216716e+10,1.812789e+11,5.029841e+11,1.408949e+11,6.375934e+11,1.253182e+11,5.133668e+12,7.642867e+10,...,1.366336e+09,8.123958e+05,9.072652e+08,2.707952e+11,4.197773e+11,3.851659e+08,3.213416e+08,2.000715e+08,3.836694e+06,0.278646
2,2,1.113490e+12,4.201177e+10,3.260794e+11,6.821171e+11,1.198528e+12,1.788118e+11,1.351327e+11,5.429972e+12,2.857559e+11,...,3.408726e+08,1.049928e+06,1.361926e+09,5.846798e+11,1.241610e+12,6.837332e+09,2.996389e+08,3.734139e+07,2.680469e+07,0.265202
3,3,3.147097e+11,1.111971e+11,8.333452e+11,1.232689e+12,7.674505e+11,5.871446e+11,1.010700e+11,5.302901e+12,5.922449e+11,...,4.852021e+07,1.205069e+05,1.500762e+09,3.241772e+11,1.900243e+11,5.430226e+09,4.057361e+08,8.593959e+07,4.533778e+07,0.085347
4,4,8.346269e+10,6.953621e+10,1.134875e+12,8.844850e+11,8.829107e+11,7.208833e+11,1.038158e+11,5.128204e+12,9.561986e+11,...,5.513583e+08,3.101359e+06,1.651712e+09,1.762151e+11,1.060050e+11,1.049285e+09,1.026361e+07,3.764703e+07,4.970396e+07,0.071189
